In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import torch
import open_clip
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/Benchmarking/Beetlepalooza_beetles_image_only.csv"
df = pd.read_csv(file_path, sep="\t")
df = df[['ImageFilePath', 'Genus']]
df.head(2)

,ImageFilePath,Genus
0,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius
1,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius


In [4]:
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms("hf-hub:imageomics/bioclip")
tokenizer = open_clip.get_tokenizer("hf-hub:imageomics/bioclip")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

for param in model.parameters() :
    param.requires_grad = False
    

In [5]:
def extract_image_features(image_path):
    
    image = Image.open(image_path).convert("RGB")

    image_tensor = preprocess_val(image).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image_tensor)
        image_features /= image_features.norm(dim=-1, keepdim=True)

    return image_features.cpu().numpy()



def classify_image(model, image_path) :
    
    image_features = extract_image_features(image_path)
    
    predicted_label = model.predict(image_features)[0]
    
    return le.inverse_transform([predicted_label])[0]



In [6]:
X = np.vstack([extract_image_features(img) for img in tqdm(df["ImageFilePath"], desc="Processing Images ... ")])

print(f"Extracted feature shape: {X.shape}")

le = LabelEncoder()

y = le.fit_transform(df["Genus"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()


Processing Images ... : 100%|██████████| 11399/11399 [19:40<00:00,  9.66it/s]

Extracted feature shape: (11399, 512)


In [7]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef


seed = 99


models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(256, 64), activation='logistic', learning_rate='adaptive', max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


NaiveBayes                | Acc: 81.32% | Prec: 85.18% | Rec: 81.32% | F1: 82.50% | Bal Acc: 73.91% | MCC: 0.7925
LogisticRegression        | Acc: 88.55% | Prec: 87.86% | Rec: 88.55% | F1: 87.25% | Bal Acc: 51.72% | MCC: 0.8705
SVMLinear                 | Acc: 92.32% | Prec: 91.43% | Rec: 92.32% | F1: 91.50% | Bal Acc: 64.23% | MCC: 0.9133
SVMPolynomial             | Acc: 95.13% | Prec: 94.94% | Rec: 95.13% | F1: 94.71% | Bal Acc: 75.42% | MCC: 0.9450
SVMRadialBasis            | Acc: 93.99% | Prec: 93.09% | Rec: 93.99% | F1: 93.38% | Bal Acc: 70.32% | MCC: 0.9321
NearestNeighbor           | Acc: 93.68% | Prec: 93.21% | Rec: 93.68% | F1: 93.19% | Bal Acc: 76.54% | MCC: 0.9287
RandomForest              | Acc: 89.61% | Prec: 89.41% | Rec: 89.61% | F1: 88.74% | Bal Acc: 60.77% | MCC: 0.8825
MLP_Baseline              | Acc: 95.09% | Prec: 94.99% | Rec: 95.09% | F1: 94.88% | Bal Acc: 78.99% | MCC: 0.9446


In [8]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,Genus,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
1766,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Synuchus,Calathus,Synuchus,Synuchus,Synuchus,Synuchus,Synuchus,Calathus,Synuchus
2629,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Agonum,Calathus,Calathus,Calathus,Calathus,Calathus,Calathus,Calathus,Calathus


In [9]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/6.BioCLIP-linear-probing-genus.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/6.BioCLIP-linear-probing-genus-metrics.csv", index=False)